# Qdrant memory 能否加载索引

证明了：

- 嵌入是保存在向量数据库中的

## 准备

In [1]:
%%time

persist_dir = "/tmp/qdrant_my_books"
collection_name="my_books"

from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(model_name="/models/bge-small-zh-v1.5")
Settings.embed_model.max_length

CPU times: user 3.95 s, sys: 512 ms, total: 4.47 s
Wall time: 4.15 s


512

## 加载

In [2]:
%%time

from qdrant_client import QdrantClient
from qdrant_client import models

client = QdrantClient(":memory:")


if not client.collection_exists(collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=Settings.embed_model.max_length,
            distance=models.Distance.COSINE,
        ),
    )

CPU times: user 744 ms, sys: 8.05 ms, total: 752 ms
Wall time: 751 ms


In [4]:
%%time

from llama_index.core import StorageContext
from llama_index.core import load_index_from_storage
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex
from qdrant_client import models

vector_store = QdrantVectorStore(
    client=client, 
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=Settings.embed_model.max_length,
        distance=models.Distance.COSINE,
    ),
)

storage_context = StorageContext.from_defaults(
    vector_store=vector_store,
    persist_dir=persist_dir
)
index = load_index_from_storage(storage_context)

CPU times: user 122 ms, sys: 11.5 ms, total: 134 ms
Wall time: 134 ms


## 查询 - 无结果

In [5]:
%%time

retriever = index.as_retriever()
nodes = retriever.retrieve("方鸿渐")

nodes

CPU times: user 394 ms, sys: 44 ms, total: 438 ms
Wall time: 437 ms


[]